<a href="https://colab.research.google.com/github/crneubert/best-music/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SETUP**


In [2]:
!pip install pyspark
!pip install -U -q PyDrive

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["PATH"] += ":/usr/lib/jvm/java-17-openjdk-amd64/bin"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [33]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql import types as sparktypes

# import PySpark aggregate functions with underscores to avoid collision with Python sum, etc
from pyspark.sql.functions import sum as _sum, avg as _avg, count as _count
from pyspark.sql.functions import col, lit, round, month, to_date, when, expr, split, first, broadcast

from pyspark.sql.window import Window
from pyspark.sql.functions import rank, dense_rank, row_number, lag, lead

In [4]:
!wget -q https://media.githubusercontent.com/media/crneubert/best-music/refs/heads/main/data/calendar.csv
!wget -q https://media.githubusercontent.com/media/crneubert/best-music/refs/heads/main/data/listings.csv
!wget -q https://media.githubusercontent.com/media/crneubert/best-music/refs/heads/main/data/reviews.csv


In [5]:
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

sc.setLogLevel("ERROR")
sqlContext = SQLContext(sc)

/usr/local/lib/python3.12/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [6]:
listings = sqlContext.read.csv("listings.csv", header = True)
calendar = sqlContext.read.csv("calendar.csv", header = True)

# **ANALYSIS**

In [7]:
listings.show(5)

+-----+--------------------+---------+---------+-------------------+--------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|   id|                name|  host_id|host_name|neighbourhood_group| neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+-----+--------------------+---------+---------+-------------------+--------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
| 6369|Rooftop terrace r...|    13660|    Simon|          Chamartín|Hispanoamérica|40.45724| -3.67688|   Private room|   60|             1|               78| 2020-09-20|             0.58|                             1|             180|
|21853|Bright and airy room|    83531|    Abdel|        

In [9]:
calendar.show(5)

+----------+----------+---------+------+--------------+--------------+--------------+
|listing_id|      date|available| price|adjusted_price|minimum_nights|maximum_nights|
+----------+----------+---------+------+--------------+--------------+--------------+
|    167183|2021-04-15|        f|$45.00|        $45.00|             1|             5|
|      6369|2021-04-15|        t|$60.00|        $60.00|             1|          1125|
|      6369|2021-04-16|        t|$60.00|        $60.00|             1|          1125|
|      6369|2021-04-17|        t|$60.00|        $60.00|             1|          1125|
|      6369|2021-04-18|        t|$60.00|        $60.00|             1|          1125|
|      6369|2021-04-19|        t|$60.00|        $60.00|             1|          1125|
|      6369|2021-04-20|        t|$60.00|        $60.00|             1|          1125|
|      6369|2021-04-21|        t|$60.00|        $60.00|             1|          1125|
|      6369|2021-04-22|        t|$60.00|        $60.00

**Cleaning Datasets**

In [10]:
listings_clean = (listings.withColumnRenamed("id", "listing_id")
                         .filter(col("room_type").isin("Shared room", "Private room", "Entire home/apt", "Hotel room")))
#Filtered to have Room Types that are not integers or unusual values



calendar_clean = (calendar.withColumn("available_boolean", when(col("available") == "t", 0).otherwise(1))
                          .groupBy("listing_id")
                          .agg(_avg("available_boolean").alias("occupancy_rate")))

#Added occupancy rate to calendar df

combo_pizza = (listings_clean.join(calendar_clean, on="listing_id")
                             .withColumn("number_of_reviews", col("number_of_reviews").cast("int"))
                             .withColumn("minimum_nights", col("minimum_nights").cast("int"))
                             .withColumn("price", col("price").cast("int"))
                             .withColumn("reviews_per_month", col("reviews_per_month").cast("int")))

#Type cast variables (that were for some reason non-integer)


+----------+--------------------+---------+--------------------+--------------------+---------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+--------------------+
|listing_id|                name|  host_id|           host_name| neighbourhood_group|  neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|      occupancy_rate|
+----------+--------------------+---------+--------------------+--------------------+---------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+--------------------+
|    167183|private house B &...|   796746|            Consuelo|           Hortaleza|        Piovera|40.45575| -3.64912|   Private room|   45|             1|               23|

**Room Type**

In [11]:
room_type = (combo_pizza.groupBy("room_type")
                        .agg(_avg("occupancy_rate").alias("avg_occupancy_rate"))
                        .orderBy("avg_occupancy_rate", ascending = False))
room_type.show()

+---------------+------------------+
|      room_type|avg_occupancy_rate|
+---------------+------------------+
|   Private room| 0.555126992439271|
|Entire home/apt| 0.529229739202593|
|    Shared room|0.5122921401780076|
|     Hotel room|0.3468134414697336|
+---------------+------------------+



Found the average occupancy rate by room type. It appears that Private rooms and entire homes/apartments have the highest occupancy. This is likely due to renters valuing privacy and seclusion.

Hotel rooms unsurprisingly have high vacancy, as hotels are likely very available and not frequently rented on AirBnB.

**What time is best to have AirBnb available to rent?**

In [26]:
best_time = (calendar.withColumn("available_boolean", when(col("available") == "t", 0).otherwise(1)) # integerized availability
                     .withColumn("date", to_date(col("date"), "yyyy-MM-dd"))
                     .withColumn("month", month(col("date"))) # date/month columns
                     .groupBy("month")
                     .agg(round(_avg("available_boolean"),2).alias("month_occ_rate")) #occupancy rate monthwise
                     .orderBy("month_occ_rate", ascending = False)
)
best_time.show()

+-----+--------------+
|month|month_occ_rate|
+-----+--------------+
|    4|          0.61|
|    3|           0.6|
|    2|           0.6|
|    1|          0.59|
|   12|          0.58|
|   11|          0.58|
|   10|          0.54|
|    5|           0.5|
|    9|          0.48|
|    8|          0.48|
|    7|          0.45|
|    6|          0.44|
+-----+--------------+



We found that occupancy rate is the highest from in April, and the pattern continues 'backwards' through the year, with May having a surge above the beginning of the summer months, potentially because some April stays bled into May. We assume this pattern is the case because people prefer to be closer to the sea in the summer months, because Madrid is inland and not next to a beach.

**Is there a correlation between Occupancy Rate and Listing Prices?**

In [13]:
print(combo_pizza.corr("occupancy_rate", "price"))


0.016470467622479442


Next we looked at correlation between occupancy rate and price, which we assumed would have a positive correlation because when properties are in more demand, the suppliers can up the prices. However, we found no correlation.

**Lets take a look at Occupancy Rates in each Neighborhood**

In [27]:
neighborhood = (combo_pizza.groupBy("neighbourhood")
                           .agg(round(_avg("occupancy_rate"), 2).alias("avg_occupancy_rate"),
                                _count("*").alias("listing_per_neighborhood"))
                           .orderBy("listing_per_neighborhood", ascending = False)
                           .filter(col("listing_per_neighborhood") > 200))
neighborhood.show()

+----------------+------------------+------------------------+
|   neighbourhood|avg_occupancy_rate|listing_per_neighborhood|
+----------------+------------------+------------------------+
|     Embajadores|              0.57|                    2311|
|     Universidad|              0.53|                    1867|
|         Palacio|              0.54|                    1499|
|             Sol|              0.52|                    1120|
|        Justicia|              0.53|                     948|
|          Cortes|              0.48|                     880|
|       Trafalgar|              0.53|                     370|
| Palos de Moguer|              0.57|                     337|
|            Goya|              0.56|                     296|
|       Argüelles|              0.58|                     280|
|       Recoletos|              0.54|                     274|
|Puerta del Angel|              0.57|                     271|
|      Guindalera|              0.54|                  

**Now lets see how occupancy rates look across each neighborhood group, since there are so many neighborhoods to look at**

In [28]:
neighborhood_group = (combo_pizza.groupBy("neighbourhood_group")
                           .agg(round(_avg("occupancy_rate"), 2).alias("avg_occupancy_rate"),
                                _count("*").alias("listing_per_group"))
                           .orderBy("listing_per_group", ascending = False)
                           .filter(col("listing_per_group") > 100))
neighborhood_group.show()

+--------------------+------------------+-----------------+
| neighbourhood_group|avg_occupancy_rate|listing_per_group|
+--------------------+------------------+-----------------+
|              Centro|              0.54|             8625|
|           Salamanca|              0.55|             1324|
|            Chamberí|              0.54|             1248|
|          Arganzuela|              0.61|             1102|
|              Tetuán|              0.54|              810|
|         Carabanchel|               0.5|              707|
|              Retiro|              0.55|              662|
|       Ciudad Lineal|              0.55|              649|
|  Puente de Vallecas|              0.44|              614|
|              Latina|              0.55|              605|
|           Chamartín|              0.52|              577|
|   Moncloa - Aravaca|              0.55|              553|
|San Blas - Canill...|              0.48|              490|
|           Hortaleza|              0.52

It appears that Centro dominates the other neighborhood groups in terms of sheer listing count, so it would be worthwhile to dive deeper into Centro data, because we can feel more confident about our conclusions given the large sample size.

**Let's look at the Average Price per Neighborhood in Centro, as well as the listing count to ensure that we have a large enough sample size.**

In [29]:
centro = (combo_pizza.filter(col("neighbourhood_group") == "Centro")
                     .groupBy("neighbourhood")
                     .agg(round(_avg("price"), 2).alias("Average Price Per Neighborhood"),
                          _count("*").alias("Listings Per Neighborhood"))
)
centro.show()

+-------------+------------------------------+-------------------------+
|neighbourhood|Average Price Per Neighborhood|Listings Per Neighborhood|
+-------------+------------------------------+-------------------------+
|  Universidad|                        108.45|                     1867|
|          Sol|                         128.2|                     1120|
|      Palacio|                        101.17|                     1499|
|     Justicia|                        108.44|                      948|
|       Cortes|                        196.07|                      880|
|  Embajadores|                        139.53|                     2311|
+-------------+------------------------------+-------------------------+



We will come back to this data later.

**We found the Spanish Zillow (Idealista) has data by neighborhood on square footage, bed and bath count, and price of sale in Madrid, so we wanted to look at price fairness for both renters and buyers**

In [18]:
import glob
import kagglehub

local_dir = kagglehub.dataset_download("kanchana1990/madrid-idealista-property-listings")
csv_path = os.path.join(local_dir, "idealista_madrid.csv")

centro_more = (spark.read.format("csv")
      .option("header", "true")
      .option("inferSchema", "true")
      .csv(csv_path))

100%|██████████| 580k/580k [00:00<00:00, 966kB/s]

Extracting files...


+--------------------+--------------------+--------------------+---------+-------+-----+-----+----+--------------------+--------------------+--------------+--------------------------+--------------------+
|                 url|          listingUrl|               title|       id|  price|baths|rooms|sqft|         description|             address|      typology|advertiserProfessionalName|      advertiserName|
+--------------------+--------------------+--------------------+---------+-------+-----+-----+----+--------------------+--------------------+--------------+--------------------------+--------------------+
|https://www.ideal...|https://www.ideal...|Piso en venta en ...|104027174|1920000|    3|    3| 183|Residencia única ...|   Recoletos, Madrid|         Pisos|           Promora Madrid |      Promora Madrid|
|https://www.ideal...|https://www.ideal...|Piso en venta en ...|102321942|1995000|    3|    3| 170|Preciosa reforma ...|  Castellana, Madrid|         Pisos|               Madrid MM

In [38]:
centro_clean = (centro_more.withColumn("address", split(col("address"), ",")[0]))
#Idealista had a column "address" that was of the format Neighborhood, Madrid which we only wanted the neighborhood from


centro_total = (centro.join(broadcast(centro_clean), centro_clean["address"].contains(centro["neighbourhood"]), "inner") # make broadcast explicit
                      .drop("address"))

#centro_total.show()
centro_total.explain()


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [neighbourhood#22, Average Price Per Neighborhood#1887, Listings Per Neighborhood#1889L, url#1225, listingUrl#1226, title#1227, id#1228, price#1229, baths#1230, rooms#1231, sqft#1232, description#1233, typology#1235, advertiserProfessionalName#1236, advertiserName#1237]
   +- BroadcastNestedLoopJoin BuildRight, Inner, Contains(address#2249, neighbourhood#22)
      :- HashAggregate(keys=[neighbourhood#22], functions=[avg(price#323), count(1)])
      :  +- Exchange hashpartitioning(neighbourhood#22, 200), ENSURE_REQUIREMENTS, [plan_id=3414]
      :     +- HashAggregate(keys=[neighbourhood#22], functions=[partial_avg(price#323), partial_count(1)])
      :        +- Project [neighbourhood#22, cast(price#26 as int) AS price#323]
      :           +- BroadcastHashJoin [listing_id#232], [listing_id#87], Inner, BuildLeft, false
      :              :- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, true]),fa

Here we join the local Centro dataset with the much smaller Idealista dataset. Due to the difference in size of these datasets, specifically the Idealista one being small, we decided to broadcast the Idealista dataset onto the larger Centro one. Doing this, we save runtime, because the Idealista dataset is in memory as the join is happening (map-side join).

In [42]:
centro_discovery = (centro_total.withColumn("Price Per Square Foot", round(col("price") / col("sqft"), 2))
                                .groupBy("neighbourhood")
                                .agg(round(_avg("Price Per Square Foot"), 2).alias("Price Per Square Foot"),
                                     first("Average Price Per Neighborhood").alias("Average Listing Price"))
                                .withColumn("Value Score (for the renter)", round(col("Price Per Square Foot") / col("Average Listing Price"), 2))
                                .orderBy("Value Score (for the renter)", ascending = False))
centro_discovery.show()
#centro_discovery.explain()

#Throw on occupancy rates monthly and other related data that could be good (per neighborhood)

+-------------+---------------------+---------------------+----------------------------+
|neighbourhood|Price Per Square Foot|Average Listing Price|Value Score (for the renter)|
+-------------+---------------------+---------------------+----------------------------+
|      Palacio|              8034.43|               101.17|                       79.42|
|     Justicia|              8370.16|               108.44|                       77.19|
|          Sol|              7749.63|                128.2|                       60.45|
|  Universidad|              6515.19|               108.45|                       60.08|
|       Cortes|              7123.18|               196.07|                       36.33|
|  Embajadores|              4700.23|               139.53|                       33.69|
+-------------+---------------------+---------------------+----------------------------+



First, we calculated the average price per square foot within each neighborhood of Centro, in order to gauge how expensive each neighborhood is in general. Then, we found the average listing price for each neighborhood to understand how much an airBnB would cost per night generally. Using both of these metrics, we then calculate a Value Score, where a higher number means better value for the renter, and a lower score means better value for a buyer looking to buy a property for the purpose of renting through AirBnB. It is important to notice that the Value Score can exceed 100, it is not standardized.

In [45]:
overall_avg = centro_total.agg(
    round(_avg("Average Price Per Neighborhood"), 2).alias("Overall Centro Average")
)

overall_avg.show()

+----------------------+
|Overall Centro Average|
+----------------------+
|                123.57|
+----------------------+



In [ ]:
listings.groupBy("room_type").count().show()